# Data preparation

Attribute Information:

* InvoiceNo: id транзакции. Если id начинается с буквы "c" - заказ был отменен
* StockCode: код товара
* Description: название товара
* Quantity: количество товара за транзакцию
* InvoiceDate: дата и время выставления счета. День и время, когда была сгенерирована транзакция
* UnitPrice: цена за единицу товара в фунтах стерлингов (£)
* CustomerID: id клиента
* Country: название страны, в которой проживает клиент

In [1]:
import datetime
import pandas as pd 
import numpy as np
# отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

In [2]:
df = pd.read_csv('data/online_retail.csv')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [3]:
# Датасет содержит 541909 строк и 8 колонок с признаками
df.shape

(541909, 8)

# Data Cleaning

Сперва надо изучить датасет, привести типы данных к нужным, заполнить или удалить пропущенные значения и тд.

In [4]:
# check missing values for each column 
df.isnull().sum().sort_values(ascending=False)

CustomerID     135080
Description      1454
InvoiceNo           0
StockCode           0
Quantity            0
InvoiceDate         0
UnitPrice           0
Country             0
dtype: int64

В датасете есть пропущенные значения в CustomerID и Description.
Их нечем заполнить, так как CustomerID - это уникальный идентификатор клиента, а Description - описание товара.
CustomerID очень важен для нашего анализа, так как мы будем смотреть на продажи в разрезе всех уникальных клиентов.
Поэтому, остается только удалить такие транзакции.

In [5]:
df = df.dropna()

In [6]:
# проверяем что пропусков не осталось
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [7]:
# типы данных
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    406829 non-null  object 
 1   StockCode    406829 non-null  object 
 2   Description  406829 non-null  object 
 3   Quantity     406829 non-null  int64  
 4   InvoiceDate  406829 non-null  object 
 5   UnitPrice    406829 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      406829 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 27.9+ MB


In [8]:
# CustomerID лучше перевести в int, тк это id код
df['CustomerID'] = df['CustomerID'].astype('int64')

# дату InvoiceDate переведем в Timestamp формат
df['InvoiceDate'] = pd.to_datetime(df.InvoiceDate, format='%m/%d/%Y %H:%M')

In [9]:
# транзакции с 01-12-2009 по 09-12-2011 (1 год), как и сказано в описании датасета
df['InvoiceDate'].min(), df['InvoiceDate'].max()

(Timestamp('2010-12-01 08:26:00'), Timestamp('2011-12-09 12:50:00'))

In [10]:
# из InvoiceDate можно получить дополнительную информацию о дате и времени проведения транзакций
df['year_month'] = df['InvoiceDate'].dt.to_period('M')
df['week'] = df['InvoiceDate'].dt.week
df['day'] = df['InvoiceDate'].dt.day
df['weekday'] = df['InvoiceDate'].dt.day_name()
df['hour'] = df['InvoiceDate'].dt.hour

# для удобства визуализации переведем в str
df['year_month'] = df['year_month'].astype(str)

In [11]:
# теперь все в порядке
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 0 to 541908
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    406829 non-null  object        
 1   StockCode    406829 non-null  object        
 2   Description  406829 non-null  object        
 3   Quantity     406829 non-null  int64         
 4   InvoiceDate  406829 non-null  datetime64[ns]
 5   UnitPrice    406829 non-null  float64       
 6   CustomerID   406829 non-null  int64         
 7   Country      406829 non-null  object        
 8   year_month   406829 non-null  object        
 9   week         406829 non-null  int64         
 10  day          406829 non-null  int64         
 11  weekday      406829 non-null  object        
 12  hour         406829 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(5), object(6)
memory usage: 43.5+ MB


In [12]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,year_month,week,day,weekday,hour
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,2010-12,48,1,Wednesday,8
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12,48,1,Wednesday,8
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,2010-12,48,1,Wednesday,8
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12,48,1,Wednesday,8
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12,48,1,Wednesday,8


In [13]:
# посмотрим на статистики
df.describe()

,Quantity,UnitPrice,CustomerID,week,day,hour
count,406829.000000,406829.000000,406829.000000,406829.000000,406829.000000,406829.000000
mean,12.061303,3.460471,15287.690570,30.936184,15.036128,12.737472
std,248.693370,69.315162,1713.600303,14.765548,8.653730,2.284952
min,-80995.000000,0.000000,12346.000000,1.000000,1.000000,6.000000
25%,2.000000,1.250000,13953.000000,19.000000,7.000000,11.000000
50%,5.000000,1.950000,15152.000000,34.000000,15.000000,13.000000
75%,12.000000,3.750000,16791.000000,44.000000,22.000000,14.000000
max,80995.000000,38970.000000,18287.000000,51.000000,31.000000,20.000000


Видно, что почему-то датасет содержит орицательные значения Quantity (количество товара за транзакцию), что невозможно.
Стоит посмотреть на эту анамалию:

In [14]:
df[df['Quantity'] < 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,year_month,week,day,weekday,hour
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527,United Kingdom,2010-12,48,1,Wednesday,9
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311,United Kingdom,2010-12,48,1,Wednesday,9
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548,United Kingdom,2010-12,48,1,Wednesday,10
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom,2010-12,48,1,Wednesday,10
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom,2010-12,48,1,Wednesday,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397,United Kingdom,2011-12,49,9,Friday,9
541541,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498,United Kingdom,2011-12,49,9,Friday,10
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311,United Kingdom,2011-12,49,9,Friday,11
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315,United Kingdom,2011-12,49,9,Friday,11


Скорее всего это ошибка при заполнении базы данных. У этих транзакций все признаки кроме Quantity не вызывавют подозрений.
Мы можем либо удалить эти транзакции, либо исправить значения Quantity:

In [15]:
df['Quantity'] = df['Quantity'].abs()

In [16]:
# теперь все в порядке
df.describe()

,Quantity,UnitPrice,CustomerID,week,day,hour
count,406829.000000,406829.000000,406829.000000,406829.000000,406829.000000,406829.000000
mean,13.412279,3.460471,15287.690570,30.936184,15.036128,12.737472
std,248.624170,69.315162,1713.600303,14.765548,8.653730,2.284952
min,1.000000,0.000000,12346.000000,1.000000,1.000000,6.000000
25%,2.000000,1.250000,13953.000000,19.000000,7.000000,11.000000
50%,5.000000,1.950000,15152.000000,34.000000,15.000000,13.000000
75%,12.000000,3.750000,16791.000000,44.000000,22.000000,14.000000
max,80995.000000,38970.000000,18287.000000,51.000000,31.000000,20.000000


In [17]:
# Добавим признак total_price - общая сумма покупки
df['total_price'] = df['Quantity'] * df['UnitPrice']

In [18]:
# сохраним предобработанный датасет
df.to_csv('data/online_retail_processed.csv', sep='\t', index=False)